# Introduction à MT avec OpenNMT.

![OPENNMT](https://avatars.githubusercontent.com/u/23035727?s=200&v=4)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RobotsMali-AI/rmai/blob/master/Introduction_MT_OpenNMT.ipynb)

Présenté par:

- Sebastien Diarra, Michael Leventhal

Procédure :

- IndabaX Mali 2022

## Agenda
- À savoir
- Introduction
- Prérequis
- Configuration de l'Environnement
- Préparation des données
- Architecture du modèle
- Entraînement
- Essai & Évaluation des performances [BLEU, Tensorboard]
- Questions et Réponses


## Buts et objectifs
- Pour que vous compreniez le MT. Pipeline
- Apprenez à configurer l'environnement OpenNMT
- Pour vous intéresser à notre initiative Bayɛlɛmabaga & MT pour les langues locales

**REMARQUES IMPORTANTES** :

- Cette pratique est destinée à ceux qui n'ont aucune expérience en MT ou ML d'ailleurs.
- La pratique est davantage axée sur l'action plutôt que sur le récit.
- J'espère que vous acheverez cette session avec un désir de contribuer au travail de MT pour le Mali.
- Vous n'avez pas besoin d'être un expert pour contribuer. Vous pouvez contribuer Data !

*NB* : Joignez-vous à quelqu'un avec un ordinateur si vous n'avez pas apporté le vôtre.

## Introduction

Dans cette pratique, nous découvrirons OpenNMT et comment entrainer un modèle de traduction automatique bilingue pour le bambara - français. Ceci est un tutoriel d'introduction qui vous permettra d'acquérir un peu d'expérience pratique. Une approche pratique est fortement recommandée, tous les participants sont encouragés à suivre. Le code source de cet atelier peut être exécuté via Colab à l'URL suivante :

[https://github.com/robotsmali-ai/rmai/blob/master/Introduction_MT_OpenNMT](https://colab.research.google.com/github/RobotsMali-AI/rmai/blob/master/Fr_Introduction_MT_OpenNMT.ipynb)


### [OpenNMT](https://opennmt.net/)

"OpenNMT - est un écosystème open source pour la traduction automatique neuronale et l'apprentissage des séquences neuronales"

C'est un framework, ou toolkit, qui permet d'entrainer un modèle de langage.

OpenNMT est pour :
- Débutants
- Amateur
- Experts

### MT (Traduction Automatique)

Processus automatisé de traduction d'une langue à une autre par machine sans intervention humaine. La traduction est généralement effectuée par un programme informatique. Il existe différents types de MT :

- MT basée sur des règles : règles de grammaire et de langue utilisées comme base pour un logiciel pour effectuer la traduction.

![RULE-IMAGE](https://i.ibb.co/7XbQV0H/grammar-rules.png)

- SMT : MT statistique : apprentissage basé sur la distribution de probabilité.


<div>
<img alt="SMT" width="300"; height="250" src="https://www.researchgate.net/profile/Andy-Way/publication/220418877/figure/fig5/AS:669019598245898@1536518110475/A-Statistical-Machine-Translation-System-adapted-from-Figure-1-in-Brown-et-al-13.png" />

<sub>Courtesy of: https://www.researchgate.net/publication/220418877_Hybrid_data-driven_models_of_machine_translation</sub>
</div>

- NMT : Neural Machine Translation : Apprentissage basé sur les réseaux de neurones artificiels.

**NMT : c'est ce qui nous intéresse dans cet atelier**

### NMT

Est l'utilisation de techniques d'apprentissage en profondeur (réseaux de neurones) pour entrainer un modèle de traduction. Cette approche est devenue l'approche dominante pour faire de la traduction automatique. NMT s'est avéré très efficace dans la traduction automatique ces dernières années.

<div>
<img width="300" height="250" src="https://1.cms.s81c.com/sites/default/files/2021-01-06/ICLH_Diagram_Batch_01_03-DeepNeuralNetwork-WHITEBG.png">
</div>

OpenNMT -- nous aide à expérimenter rapidement la NMT, ainsi qu'à créer des systèmes/solutions de MT en moins de temps.

NMT englobe les deux autres approches dans une certaine mesure.

**NB** : ***Assistez à l'atelier NLP le samedi pour apprendre ces concepts en détail.***

## Prérequis
- Compréhension de la CLI Unix (aide)
- Compréhension de Python (JupyterNB)
- Les concepts de base de ML (tels que Pipeline, Models, NN) aident mais ne sont pas obligatoires



## Configuration de l'Environnement

### Paquets (Modules)

- `OpenNMT-py` : paquet opennmt principal (version pytorch)
- `rmaipkg` : Package pour les ensembles de données et les modèles de RobotsMaliAI (Later)
- `Daba` (non requis) : une version modifiée de [daba](https://github.com/maslinych/daba)

In [ ]:
%pip install OpenNMT-py subword_nmt sentencepiece # Main OpenNMT Package (No Need for subword_nmt sentencepiece)
%pip install --no-cache-dir https://github.com/RobotsMali-AI/rmai/releases/download/0.0.4/rmaipkg-0.0.4.tar.gz # RobotsMaliAI's Datasets and Models
%pip install -U https://github.com/s7d11/daba/releases/download/v0.0.1-alpha/daba-0.9.2.tar.gz # Non-UI Version of Daba
%pip install sacrebleu # Redundant comes with OpenNMT-py

### Configuration de l'espace de travail (Google Colab)

- Qu'est-ce que Colab ?

- Local vs Google Drive
    - Experimentation vs Persistence de Données

In [ ]:
import os
import pathlib
# TODO: validate that user works on Colab

# Select your environment
WORK_ENV = 0 # 0=Local 1=Drive

CWD = os.getcwd()
WORK_DIR_Lo = CWD+"/onmt" if not os.path.exists("/content") else "/content/onmt" #"/content/onmt"

if(WORK_ENV):
    if os.path.exists("/content"):
        from google.colab import drive
        WORK_DIR_Dr = "/content/drive/MyDrive/onmt"
        drive.mount('/content/drive') # Mounting your Google drive
    else:
        print("Colab not detected. Reverting to local environment")
        WORK_DIR_Dr = WORK_DIR_Lo

WORK_DIR = WORK_DIR_Dr if WORK_ENV else WORK_DIR_Lo
DATA_DIR = f"{WORK_DIR}/data"

if(not os.path.exists(WORK_DIR)):
    !mkdir -p $DATA_DIR
    print(f"Environment Created: {WORK_DIR}")
else:
    print("Environment already exists...")

os.chdir(WORK_DIR) # Navigating to Work-environment

### Activer le GPU (uniquement Colab)

- Édition > Paramètres du notebook > GPU > Enregistrer
- Exécution > GPU


**IMPORTANT** : Gardez le GPU détaché jusqu'à ce que vous fassiez quelque chose qui nécessite des GPU (c'est-à-dire de l'entrainement)

In [ ]:
# GPU's information

!nvidia-smi 

## Préparation des données

`rmaipkg` a des textes Bambara-Français parallèles que nous utiliserons pour cette tutoriels. Nous pouvons facilement integrer cela en tant que source de données.

Sachez à l'avance : ***Le texte n'est pas entièrement nettoyé si vous rencontrez des problèmes, il est préférable de générer un échantillon différent***

- `parallel` : module pour les textes parallèles de RobotsMali
- `parallel.get_text` :
    - retourne `max_len` nombre de lignes
    - `randomize` lorsqu'il est défini, mélange les textes. Bon pour générer un ordre différent pour les textes à chaque exécution.

In [ ]:
# Import the 'parallel' module from rmaipkg
from rmai.datasets.text import parallel

texts = parallel.get_text(
    max_len=50000, randomize=True) # Load texts in memory

# Show first 10 lines
print(texts[:10])

- `parallel.random_split` : divise l'objet texte en ratios pour l'apprentissage et la validation de l'apprentissage, il renvoie un type de tuple.
    - Le premier argument est les "textes"
    - Le deuxième argument est le taux auquel c'est divisé

In [ ]:
train, valid = parallel.random_split(texts, 90) # Split Data into training and validation 80/20

print(
    "Split correctly: ", 
    len(train)+len(valid) == len(texts)) # Sanity Chec

*Tâche 1* Votre tâche ici est de créer les variables suivantes. Le contenu associé avec chaque variable sera les textes d'apprentissage ou de validation dans chaque langue.

Astuce : Les textes sont dans les tableaux de tuples train et valid, le bambara précède le français dans chaque tuple.

Si votre Python n'est pas encore à l'hauteur, vous trouverez la solution dans la cellule ci-dessous, vous pouvez l'exécuter directement.

Les variables à créer sont :
- trainbam: train set bambara
- trainfra: train set francais
- validbam: valid set bambara
- validfra: valid set francais

In [ ]:
# CODE ou PSEUDOCODE ICI

In [ ]:
#@title
# Task I - Solution
x_extractor = lambda x, dset: [i[x] for i in dset]

trainbam = x_extractor(0, train)
trainfra = x_extractor(1, train)
validbam = x_extractor(0, valid)
validfra = x_extractor(1, valid)

- `x_extractor` est une fonction lambda pour la tâche de séparer les valeurs.

In [ ]:
# Pour améliorer les résultats - éliminer les caractères superflues des textes

import string

def clean_lines(lines):
  nlines = []
  for i in lines:
    for j in string.punctuation + "«»":
      if(not j in r"!.:?()[]"):
        i = i.replace(j, " ")
    nlines.append(i)
  return " ".join("".join(nlines).strip().split())

trainbam = [clean_lines(i) for i in trainbam]
trainfra = [clean_lines(i) for i in trainfra]
validbam = [clean_lines(i) for i in validbam]
validfra = [clean_lines(i) for i in validfra]


- Écrivons maintenant nos données dans des fichiers réels sur le système. `rmaipkg` a une méthode appelée `write_to` qui le fait pour nous.

In [ ]:
parallel.write_to(lines=trainbam, name="src-train", path=DATA_DIR)
parallel.write_to(lines=trainfra, name="tgt-train", path=DATA_DIR)
parallel.write_to(lines=validbam, name="src-val", path=DATA_DIR)
parallel.write_to(lines=validfra, name="tgt-val", path=DATA_DIR)


Maintenant que nous avons écrit nos données et que nous avons un ensemble de données, écrivons les informations afin que le modèle puisse l'utiliser pour l'apprentissage et la validation (configuration).

In [ ]:

model_name = "bam2fr" # Nom de notre modèle
config_name = f"{WORK_DIR}/cool_config.yaml"

config = f"""

# IndabaX - Mali 2022 (Configuration)

save_data: {model_name}/run/{model_name}

overwrite: True # Toggle this for rewritting

data:  
    robotsmali:
        path_src: data/src-train.txt # DataPath
        path_tgt: data/tgt-train.txt
        transforms: []
        weight: 1
    valid:
        path_src: data/src-val.txt
        path_tgt: data/tgt-val.txt
        transforms: []
        weight: 1

# src_seq_length: 250
# tgt_seq_length: 250

src_vocab: {model_name}/run/{model_name}.vocab.src.txt
tgt_vocab: {model_name}/run/{model_name}.vocab.tgt.txt
# skip_empty_level: silent

tensorboard: true
tensorboard_log_dir: logs

"""

# Write config file
with open(config_name, "w") as fp:
    fp.write(config)


Ici, nous avons créé un fichier de configuration `cool_config.yaml` contenant des informations générales sur le jeu de données. Qu'avons-nous fait ? :

- Nous avons attribué le nom de notre modèle à une variable
- Générez un fichier de configuration avec quelques détails de base tels que :
    - où se trouvent les données
    - où générer les fichiers de vocabulaire

Maintenant, nous devons transformer nos textes en une représentation traitable par le modèle, souvent appelée la vectorisation des textes ou la création d'un vocabulaire (en français technique, le plongement lexical). OpenNMT vous offre la fonction *onmt_build_vocab* pour cette tâche.

In [ ]:
# OpenNMT Construire le vocabulaire 

!onmt_build_vocab -c $config_name -n_sample -1 --dump_samples # -1 full corpus, bpe, sentencepiece

In [ ]:
!head bam2fr/run/bam2fr.vocab.src.txt
!tail bam2fr/run/bam2fr.vocab.src.txt

La commande `onmt_build_vocab` construit le vocabulaire nécessaire à l'apprentissage. Ici, nous l'avons appelé avec trois paramètres

- `-c config_file` : chemin du fichier de configuration à utiliser pour le processus de création de vocabulaire

- `-n_sample -1` : la construction d'un vocabulaire à l'aide du corpus complet peut remplacer **-1** par n'importe quel nombre, lorsque l'argument est omis, sa valeur par défaut est 5000.

- `-- dump_samples` : (argument facultatif) Écrire des échantillons lors de la construction du vocabulaire, trouvés dans `model_name/run/model_name`

On n'est pas obligé d'utiliser *onmt_build_vocab* pour la construction du vocabulaire, nous pouvons appliquer notre propre algorithme pour arriver à la même fin. Souvent, cela est nécessaire afin d'arriver à des résultats optimaux. Ci-dessous nous remplaçons *onmt_build_vocab* avec une fonction de tokenisation.

In [ ]:
# Custom Tokenization (rm-daba-tokenizer) 
# - Shown as example, CELL can be skipped

from rmai.utils import daba

dabautil = daba.DabaUtils()
dabautil.tokenize_line(trainbam+validbam)[:20]

[`Daba`](https://github.com/maslinych/daba) est un ensemble logiciel incroyable développé par des linguistes de l'INALCO. Daba est une boîte à outils pour les linguistes, et par conséquent pour nous, les gens des sciences informatiques qui cherchons à travailler avec la langue bambara.

- Si vous cherchez à travailler avec Bambara, nous vous le recommandons fortement.


Mais, pour la suite de cet atelier, nous allons utiliser le vocabulaire que nous avons créé avec OpenNMT.

## Apprentissage


#### Architecture d'un modèle

Nous "concevons" le modèle à travers la configuration que nous avons générée précédemment. Dans la cellule de code ci-dessous, nous fixons tous les paramètres de notre modèle.

*Bien que le fichier de configuration puisse subir de nombreuses modifications afin d'optimiser les résultats,* nous limiterons les changements à quelques options, car nous n'avons pas assez de temps pour discuter tous. Le sujet est vaste.

Vous pouvez appeler cette configuration comme ingrédients de la sorcière.

- save_model : est le chemin dans lequel le point de contrôle de notre modèle est enregistré.
- save_checkpoint_steps : X, à chaque étape X de l'entraînement, un point de contrôle est enregistré.
- train_steps : Nombre d'étapes d'entraînement
- valid_steps : exécuter la validation, toutes les X étapes
- warmup_step : étape permettant d'ajuster le taux d'apprentissage après le début de l'entraînement.
- GPU : basculer si vous utilisez GPU ou CPU

In [ ]:
training_steps = 500 # 1000 # Number of steps for the training
save_freq =  int(training_steps / 2) #100 # Checkpoint saving steps
valid_steps = int(training_steps/ 4) # 50 # Steps to validate training perfomance
warmup_step = int(training_steps/ 4) #125 # Warmup 1/4th of training total
GPU = 0 # 0: CPU or 1:GPU

config += f"""
save_model: {model_name}/{model_name}
save_checkpoint_steps: {save_freq}
keep_checkpoint: 1 # Better for this activity
seed: 1234
train_steps: {training_steps}
valid_steps: {valid_steps}
report_every: {valid_steps}
warmup_steps: {warmup_step}
"""

if(GPU):
    config += """
world_size: 1 # CPU
gpu_ranks: [0] # Change to number of GPU if more than 1 GPU
"""

Bon à savoir : 2-layer LSTM configuration automatique, en bas Il s'agit d'une architecture de modèle de transformateur (transformer).

Inspiré de :
- [https://github.com/OpenNMT/OpenNMT-py/blob/master/config/config-transformer-base-1GPU.yml](https://github.com/OpenNMT/OpenNMT-py/blob/master/config/config-transformer-base-1GPU.yml)
- Travail effectué par [A. A. Tapo](https://github.com/israaar)

- **Toggle**: pour transformateur

In [ ]:
#@title
# Transformer Configuration

transformer = 0 # Set to 1
if (transformer):
  config += """

batch_type: "tokens"
batch_size: 256
valid_batch_size: 64
max_generator_batches: 2

# Optimization
model_dtype: "text"
optim: "adam"
learning_rate: 1
decay_method: "noam"
adam_beta2: 0.998
max_grad_norm: 0
label_smoothing: 0.1
param_init: 0
param_init_glorot: true
normalization: "tokens"

# Model
encoder_type: transformer
decoder_type: transformer
position_encoding: true
enc_layers: 6
dec_layers: 6
heads: 4
hidden_size: 512
word_vec_size: 512
transformer_ff: 2048
dropout_steps: [0]
dropout: [0.1]
attention_dropout: [0.1]

"""


*Tâche 2* Votre tâche ici est de réécrire le fichier de configuration avec la configuration nouvelle. 

Astuce : Le fichier est *config_name*, les paramètres nouveaux se trouvent dans la variable *config*, en Python on remplace le contenu d'un fichier avec l'indication *w*.

Si votre Python n'est pas encore à l'hauteur, vous trouverez la solution dans la cellule ci-dessous, vous pouvez l'exécuter directement.

In [ ]:
# CODE ou PSEUDOCODE ICI

In [ ]:
#@title

with open(config_name, "w") as fp:
    fp.write(config)


- **NOW LET THE MAGIC BEGIN (Place à la magie !)**

Nous allons exécuter notre expérience, mais avant de lancer la commande d'apprentissage, nous allons démarrer tensorboard. ***Tensorboard*** nous aide à visualiser ce qui se passe avec notre modèle en entrainement.

**Important** : Si vous obtenez une erreur `403`. Activez Cookie / Autorisez le navigateur à afficher le Tensorboard.

In [ ]:
%load_ext tensorboard

%tensorboard --logdir bam2fr/logs

Le `onmt_train` prend le fichier de configuration comme argument. Espérons que si nous avons tout fait avant, l'entrainenment devrait commencer sans aucun problème. La durée de la formation dépend des étapes de l'entrainement, de la puissance de calcul et de data_size.

In [ ]:
!onmt_train -config $config_name

## Essai & Évaluation des performances [BLEU, Tensorboard]

### Tâche 3
Vous allez créer vos propres données pour tester la performance du modèle. Il s'agit de préparer les phrases en bambara, étant le texte source ou la question, et de préparer la traduction de chaque phrase en français, étant la bonne réponse. Puis, vous allez soumettre vos textes au modèle et vous allez comparer ce que le modèle donne pour réponse à la bonne réponse qu'un expert (vous) a préparé.

- Créer les fichiers de test (les phrases équivalentes dans les deux langues)
    - Créez un nouveau fichier avec le nom *mbam.txt* dans le répertoire `data` puis écrivez quelques lignes de bambara dans le fichier.
    - Créez un autre fichier nommé *mfr.txt* dans le répertoire `data` puis écrivez les traductions françaises des phrases dans le fichier texte de bambara dans le même ordre.
    
*Astuce* : Beaucoup de Maliens bambaraphone n'ont pas l'habitude d'écrire le Bambara. C'est triste, mais vrai ! Si c'est votre cas, vous pourriez utiliser Google traduction (https://translate.google.com/?sl=fr&tl=bm&op=translate) pour créer vos phrases en Bambara depuis les phrases que vous écrivez en français.

*Astuce 2* : Si votre français n'est pas non plus trop recommandable, vous pouvez copier des phrases de ce notebook à Google traduction.

Si vous vous sentez trop paresseux ou sans inspiration pour même copier les phrases, utilisez donc notre kit de test en exécutant la cellule ci-dessous.

In [ ]:
#@title
# Test Set
bamf = "https://raw.githubusercontent.com/RobotsMali-AI/rmai/master/mbam.txt"
fraf = "https://raw.githubusercontent.com/RobotsMali-AI/rmai/master/mfr.txt"

!wget {bamf} {fraf} -P data # Bam-Fra


La commande `onmt_translate` est utilisée pour effectuer la traduction avec le modèle formé. En langage de l'apprentissage automatique, l'opération s'appelle *inférence*. Ici, nous utilisons cinq paramètres :
- `model` : un fichier de point de contrôle de modèle
- `src` : notre fichier de test la langue source
- `output` : où nous voulons écrire les prédictions de notre modèle
- `gpu` : nombre de GPU sur lesquels exécuter
- `verbose` : affiche les scores par prédiction

In [ ]:
ckpt_step = training_steps

tsrc = "mbam.txt" # Change this to your bambara test file

selected_model_name = f"{model_name}_step_{ckpt_step}.pt"
output = f"{model_name}/pred_{model_name}_{ckpt_step}.txt"

!onmt_translate -model $model_name/$selected_model_name -src data/$tsrc -output $output # -gpu -1 -verbose

In [ ]:
!cat data/$tsrc

**Ta-da! Let's see how our model did, now. Voyons nos résultats. **

In [ ]:
!cat $output

Le score de Bleu `(BiLingual Evaluation Understudy)` est une mesure automatique pour évaluer la traduction automatique. C'est utile, car cela nous permet d'avoir une idée générale de la façon dont notre modèle fonctionne si nous travaillons avec un modèle plus grand et une taille de données plus grande.

Dans notre cas, nous utiliserons notre propre traduction comme référence pour déterminer les performances de notre modèle.

In [ ]:
reference = "data/mfr.txt"

!sacrebleu $reference -i $output -m bleu -b -w 4

Prochaine étape : revenez en arrière et ajustez les paramètres pour votre propre personnalisation.

>>> **Merci de nous avoir rejoint. Assistez à l'atelier NLP le samedi pour en savoir plus sur le NLP, et le ASR.**

<div>

<img width="250" src="https://sp-ao.shortpixel.ai/client/to_webp,q_glossy,ret_img/https://indabax.robotsmali.org/wp-content/uploads/2022/11/robotsmali.png" />

</div>